# Efficient Yelp API Calls

- use the Yelp API to search your favorite city for a cuisine type of your choice.

- Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

## Import libraries

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## API login creds

In [2]:
# Load API Credentials
with open('/Users/alicia/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

## Defining terms of search

In [5]:
# set our API call parameters and filename before the first call
LOCATION = 'Tallahasse, FL,32302'
TERM = 'Sushi'

In [6]:
# Specifying JSON_FILE filename 
JSON_FILE = f"Data/results_in_progress_sushi.json"
JSON_FILE

'Data/results_in_progress_sushi.json'

In [7]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_sushi.json not found. Saving empty list to file.


In [8]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
## How many results total?
total_results = results['total']
total_results

5

In [11]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

5

In [14]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

1

In [15]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [16]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/1 [00:00<?, ?it/s]

## Making dataframe

In [18]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,3ZgIHvi0Qjba_VHBC8cGKw,chuck-s-fish-tallahassee,Chuck’s Fish,https://s3-media3.fl.yelpcdn.com/bphoto/kgJ24l...,False,https://www.yelp.com/biz/chuck-s-fish-tallahas...,39,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 30.4408853, 'longitude': -84.279891}",[],"{'address1': '224 E College Ave', 'address2': ...",18505977506,(850) 597-7506,99.223465,NaN
1,taxvnCs9oFC1VQJ86TW9Bg,straw-hat-poke-tallahassee,Straw Hat Poke,https://s3-media1.fl.yelpcdn.com/bphoto/4O6pcE...,False,https://www.yelp.com/biz/straw-hat-poke-tallah...,20,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...",4.5,"{'latitude': 30.43638, 'longitude': -84.29203}","[delivery, pickup]","{'address1': '631 W Madison St', 'address2': '...",18502222012,(850) 222-2012,1252.007668,NaN
2,bxQ8E1S0-2721rlxOoB54w,roboto-tokyo-grill-tallahassee-2,Roboto Tokyo Grill,https://s3-media2.fl.yelpcdn.com/bphoto/snMiO-...,False,https://www.yelp.com/biz/roboto-tokyo-grill-ta...,1,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",5.0,"{'latitude': 30.43459315789246, 'longitude': -...","[delivery, pickup]","{'address1': '825 Railroad Ave', 'address2': '...",18509992700,(850) 999-2700,1084.308303,NaN
3,-GJBOfd2izzd1359hBDDRQ,andrews-downtown-tallahassee,Andrew's Downtown,https://s3-media4.fl.yelpcdn.com/bphoto/UvZS95...,False,https://www.yelp.com/biz/andrews-downtown-tall...,186,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,"{'latitude': 30.44018, 'longitude': -84.28231}",[delivery],"{'address1': '228 S Adams St', 'address2': '',...",18502223444,(850) 222-3444,222.809954,$$
4,dfkdeh_Y2e8dUdHFg9vIKw,greenwise-market-tallahassee,GreenWise Market,https://s3-media1.fl.yelpcdn.com/bphoto/8KmFSH...,False,https://www.yelp.com/biz/greenwise-market-tall...,43,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.5,"{'latitude': 30.43487, 'longitude': -84.29161}",[],"{'address1': '625 W Gaines St', 'address2': ''...",18505771915,(850) 577-1915,1256.333835,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
5,3ZgIHvi0Qjba_VHBC8cGKw,chuck-s-fish-tallahassee,Chuck’s Fish,https://s3-media3.fl.yelpcdn.com/bphoto/kgJ24l...,False,https://www.yelp.com/biz/chuck-s-fish-tallahas...,39,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 30.4408853, 'longitude': -84.279891}",[],"{'address1': '224 E College Ave', 'address2': ...",18505977506,(850) 597-7506,99.223465,NaN
6,taxvnCs9oFC1VQJ86TW9Bg,straw-hat-poke-tallahassee,Straw Hat Poke,https://s3-media1.fl.yelpcdn.com/bphoto/4O6pcE...,False,https://www.yelp.com/biz/straw-hat-poke-tallah...,20,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...",4.5,"{'latitude': 30.43638, 'longitude': -84.29203}","[delivery, pickup]","{'address1': '631 W Madison St', 'address2': '...",18502222012,(850) 222-2012,1252.007668,NaN
7,bxQ8E1S0-2721rlxOoB54w,roboto-tokyo-grill-tallahassee-2,Roboto Tokyo Grill,https://s3-media2.fl.yelpcdn.com/bphoto/snMiO-...,False,https://www.yelp.com/biz/roboto-tokyo-grill-ta...,1,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",5.0,"{'latitude': 30.43459315789246, 'longitude': -...","[delivery, pickup]","{'address1': '825 Railroad Ave', 'address2': '...",18509992700,(850) 999-2700,1084.308303,NaN
8,-GJBOfd2izzd1359hBDDRQ,andrews-downtown-tallahassee,Andrew's Downtown,https://s3-media4.fl.yelpcdn.com/bphoto/UvZS95...,False,https://www.yelp.com/biz/andrews-downtown-tall...,186,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,"{'latitude': 30.44018, 'longitude': -84.28231}",[delivery],"{'address1': '228 S Adams St', 'address2': '',...",18502223444,(850) 222-3444,222.809954,$$
9,dfkdeh_Y2e8dUdHFg9vIKw,greenwise-market-tallahassee,GreenWise Market,https://s3-media1.fl.yelpcdn.com/bphoto/8KmFSH...,False,https://www.yelp.com/biz/greenwise-market-tall...,43,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.5,"{'latitude': 30.43487, 'longitude': -84.29161}",[],"{'address1': '625 W Gaines St', 'address2': ''...",18505771915,(850) 577-1915,1256.333835,$$


In [36]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_crab_cakes.csv.gz', compression='gzip',index=False)